# Day 16: SaaS Health Metrics Dashboard - Data Verification

This notebook generates all dashboard visualizations in Python to verify the numbers before creating them in Metabase.

**Purpose:** Compare these results with your BigQuery data to ensure accuracy.

---

## Setup: Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set figure size defaults
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

In [ ]:
# Load all data files
data_dir = Path('data')

# Read CSVs
df_kpis = pd.read_csv(data_dir / 'day06_dashboard_kpis.csv')
df_mrr_summary = pd.read_csv(data_dir / 'day06_mrr_summary.csv')
df_retention = pd.read_csv(data_dir / 'day06_retention_curves.csv')
df_churn_cohort = pd.read_csv(data_dir / 'day06_churn_by_cohort.csv')
df_customer_health = pd.read_csv(data_dir / 'day06_customer_health.csv')
df_customers = pd.read_csv(data_dir / 'day06_customers.csv')
df_subscriptions = pd.read_csv(data_dir / 'day06_subscriptions.csv')
df_mrr_movements = pd.read_csv(data_dir / 'day06_mrr_movements.csv')

print("✅ All data loaded successfully!")
print(f"\n📊 Dataset sizes:")
print(f"  - KPIs: {len(df_kpis)} rows")
print(f"  - MRR Summary: {len(df_mrr_summary)} rows")
print(f"  - Retention Curves: {len(df_retention)} rows")
print(f"  - Churn by Cohort: {len(df_churn_cohort)} rows")
print(f"  - Customer Health: {len(df_customer_health)} rows")
print(f"  - Customers: {len(df_customers)} rows")
print(f"  - Subscriptions: {len(df_subscriptions)} rows")
print(f"  - MRR Movements: {len(df_mrr_movements)} rows")

---
## Section 1: Business Health Baseline (4 KPIs)

These are the executive summary metrics that appear at the top of the dashboard.

In [ ]:
# Card 1.1: Current MRR
current_mrr = df_kpis['current_mrr'].iloc[0]
print(f"💰 Current MRR: ${current_mrr:,.2f}")

In [ ]:
# Card 1.2: Churn Rate (%)
churn_rate = df_kpis['overall_churn_rate_pct'].iloc[0]
print(f"📉 Churn Rate: {churn_rate:.2f}%")

In [ ]:
# Card 1.3: Active Customers
active_customers = df_kpis['active_customers'].iloc[0]
print(f"👥 Active Customers: {active_customers:,}")

In [ ]:
# Card 1.4: Healthy Customers %
healthy = df_kpis['healthy_customers'].iloc[0]
at_risk = df_kpis['at_risk_customers'].iloc[0]
critical = df_kpis['critical_customers'].iloc[0]
total = healthy + at_risk + critical
healthy_pct = (healthy / total) * 100

print(f"✅ Healthy Customers: {healthy_pct:.1f}%")
print(f"   - Healthy: {healthy}")
print(f"   - At Risk: {at_risk}")
print(f"   - Critical: {critical}")
print(f"   - Total: {total}")

In [ ]:
# Summary: All 4 KPIs together
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Section 1: Business Health Baseline (4 KPIs)', fontsize=16, fontweight='bold')

# KPI 1: Current MRR
axes[0, 0].text(0.5, 0.5, f'${current_mrr:,.0f}', 
                ha='center', va='center', fontsize=36, fontweight='bold', color='#2ecc71')
axes[0, 0].text(0.5, 0.2, 'Current MRR', ha='center', va='center', fontsize=14, color='gray')
axes[0, 0].axis('off')

# KPI 2: Churn Rate
axes[0, 1].text(0.5, 0.5, f'{churn_rate:.1f}%', 
                ha='center', va='center', fontsize=36, fontweight='bold', color='#e74c3c')
axes[0, 1].text(0.5, 0.2, 'Churn Rate', ha='center', va='center', fontsize=14, color='gray')
axes[0, 1].axis('off')

# KPI 3: Active Customers
axes[1, 0].text(0.5, 0.5, f'{active_customers:,}', 
                ha='center', va='center', fontsize=36, fontweight='bold', color='#3498db')
axes[1, 0].text(0.5, 0.2, 'Active Customers', ha='center', va='center', fontsize=14, color='gray')
axes[1, 0].axis('off')

# KPI 4: Healthy Customers %
axes[1, 1].text(0.5, 0.5, f'{healthy_pct:.1f}%', 
                ha='center', va='center', fontsize=36, fontweight='bold', color='#2ecc71')
axes[1, 1].text(0.5, 0.2, 'Healthy Customers', ha='center', va='center', fontsize=14, color='gray')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

---
## Section 2: Growth Trajectory (2 Cards)

These visualizations show MRR growth trends over time.

In [ ]:
# Card 2.1: MRR Growth Over Time (Stacked Area Chart)
# Parse month as datetime
df_mrr_summary['month'] = pd.to_datetime(df_mrr_summary['month'])
df_mrr_summary = df_mrr_summary.sort_values('month')

# Display data
print("📊 MRR Summary Data:")
print(df_mrr_summary[['month', 'total_mrr_growth', 'new_mrr', 'expansion_mrr', 'contraction_mrr', 'churned_mrr']].head(10))

In [ ]:
# Plot Card 2.1: MRR Growth Over Time (Stacked Area Chart)
fig, ax = plt.subplots(figsize=(14, 6))

# Create stacked area chart
ax.fill_between(df_mrr_summary['month'], 0, df_mrr_summary['new_mrr'], 
                alpha=0.7, label='New MRR', color='#2ecc71')
ax.fill_between(df_mrr_summary['month'], df_mrr_summary['new_mrr'], 
                df_mrr_summary['new_mrr'] + df_mrr_summary['expansion_mrr'], 
                alpha=0.7, label='Expansion MRR', color='#3498db')
ax.fill_between(df_mrr_summary['month'], 
                df_mrr_summary['new_mrr'] + df_mrr_summary['expansion_mrr'], 
                df_mrr_summary['new_mrr'] + df_mrr_summary['expansion_mrr'] - df_mrr_summary['contraction_mrr'], 
                alpha=0.7, label='Contraction MRR', color='#f39c12')
ax.fill_between(df_mrr_summary['month'], 
                df_mrr_summary['new_mrr'] + df_mrr_summary['expansion_mrr'] - df_mrr_summary['contraction_mrr'], 
                df_mrr_summary['total_mrr_growth'], 
                alpha=0.7, label='Churned MRR', color='#e74c3c')

# Overlay total MRR growth line
ax.plot(df_mrr_summary['month'], df_mrr_summary['total_mrr_growth'], 
        color='black', linewidth=2, label='Net MRR Growth', marker='o')

ax.set_title('Card 2.1: MRR Growth Over Time (Waterfall)', fontsize=14, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('MRR ($)', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Card 2.2: Month-over-Month Growth Rate (Line Chart)
# Calculate MoM growth rate
df_mrr_summary['growth_rate_pct'] = df_mrr_summary['growth_rate_pct']

print("📈 MoM Growth Rate Data:")
print(df_mrr_summary[['month', 'growth_rate_pct']].tail(10))

In [ ]:
# Plot Card 2.2: Month-over-Month Growth Rate
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df_mrr_summary['month'], df_mrr_summary['growth_rate_pct'], 
        color='#3498db', linewidth=2, marker='o', markersize=6)

# Add zero line
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Color positive/negative areas
ax.fill_between(df_mrr_summary['month'], 0, df_mrr_summary['growth_rate_pct'], 
                where=(df_mrr_summary['growth_rate_pct'] >= 0), alpha=0.3, color='green', label='Positive Growth')
ax.fill_between(df_mrr_summary['month'], 0, df_mrr_summary['growth_rate_pct'], 
                where=(df_mrr_summary['growth_rate_pct'] < 0), alpha=0.3, color='red', label='Negative Growth')

ax.set_title('Card 2.2: Month-over-Month Growth Rate (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Growth Rate (%)', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---
## Section 3: Cohort Retention Analysis (2 Cards)

⭐ **PRIMARY DECISION VISUAL**: Cohort retention curves show which customer cohorts are declining.

In [ ]:
# Card 3.1: Cohort Retention Curves (PRIMARY VISUAL)
# Parse cohort_month as datetime
df_retention['cohort_month'] = pd.to_datetime(df_retention['cohort_month'])

# Filter to first 12 months
df_retention_12m = df_retention[df_retention['months_since_signup'] <= 12].copy()

print("🎯 Cohort Retention Data (first 5 cohorts):")
cohorts = df_retention_12m['cohort_month'].unique()[:5]
for cohort in cohorts:
    cohort_data = df_retention_12m[df_retention_12m['cohort_month'] == cohort]
    print(f"\nCohort {cohort.strftime('%Y-%m')}:")
    print(cohort_data[['months_since_signup', 'retention_rate_pct']].to_string(index=False))

In [ ]:
# Plot Card 3.1: Cohort Retention Curves (PRIMARY VISUAL)
fig, ax = plt.subplots(figsize=(14, 8))

# Plot each cohort as a separate line
cohorts = sorted(df_retention_12m['cohort_month'].unique())
colors = sns.color_palette('husl', len(cohorts))

for i, cohort in enumerate(cohorts):
    cohort_data = df_retention_12m[df_retention_12m['cohort_month'] == cohort]
    cohort_data = cohort_data.sort_values('months_since_signup')
    
    ax.plot(cohort_data['months_since_signup'], cohort_data['retention_rate_pct'], 
            color=colors[i], linewidth=2, marker='o', markersize=4, 
            label=cohort.strftime('%Y-%m'), alpha=0.8)

ax.set_title('⭐ Card 3.1: Cohort Retention Curves (PRIMARY DECISION VISUAL)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Months Since Signup', fontsize=12)
ax.set_ylabel('Retention Rate (%)', fontsize=12)
ax.set_ylim(0, 105)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2, fontsize=8)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 DECISION INSIGHT:")
print("   - Look for cohorts with steeper decline slopes = higher churn risk")
print("   - Compare early cohorts vs recent cohorts to see if retention is improving")

In [ ]:
# Card 3.2: Churn Heatmap by Cohort × Plan Tier
# Parse cohort_month
df_churn_cohort['cohort_month'] = pd.to_datetime(df_churn_cohort['cohort_month'])

print("🔥 Churn by Cohort × Plan Tier:")
print(df_churn_cohort.head(20))

In [ ]:
# Plot Card 3.2: Churn Heatmap
# Pivot data for heatmap
pivot = df_churn_cohort.pivot_table(
    index='cohort_month', 
    columns='plan_tier', 
    values='churn_rate_pct', 
    aggfunc='mean'
)

# Format cohort labels
pivot.index = pivot.index.strftime('%Y-%m')

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd', 
            linewidths=0.5, cbar_kws={'label': 'Churn Rate (%)'}, ax=ax)

ax.set_title('Card 3.2: Churn Heatmap by Cohort × Plan Tier', fontsize=14, fontweight='bold')
ax.set_xlabel('Plan Tier', fontsize=12)
ax.set_ylabel('Cohort Month', fontsize=12)
plt.tight_layout()
plt.show()

print("\n💡 DECISION INSIGHT:")
print("   - Darker red = higher churn risk for that cohort × plan combination")
print("   - Target interventions to high-churn segments")

---
## Section 4: Customer Health Alerts (2 Cards)

These cards identify at-risk customers for proactive intervention.

In [ ]:
# Card 4.1: At-Risk Customer Distribution (Pie Chart)
health_distribution = df_customer_health['health_status'].value_counts()

print("🚨 Customer Health Distribution:")
print(health_distribution)
print(f"\nTotal customers: {health_distribution.sum()}")

In [ ]:
# Plot Card 4.1: At-Risk Customer Distribution
fig, ax = plt.subplots(figsize=(10, 8))

colors = {'Healthy': '#2ecc71', 'At Risk': '#f39c12', 'Critical': '#e74c3c'}
pie_colors = [colors.get(status, 'gray') for status in health_distribution.index]

wedges, texts, autotexts = ax.pie(health_distribution.values, 
                                    labels=health_distribution.index,
                                    autopct='%1.1f%%',
                                    startangle=90,
                                    colors=pie_colors,
                                    textprops={'fontsize': 12, 'weight': 'bold'})

# Make percentage text white for visibility
for autotext in autotexts:
    autotext.set_color('white')

ax.set_title('Card 4.1: At-Risk Customer Distribution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Card 4.2: Top 10 Critical Customers (Table)
critical_customers = df_customer_health[
    df_customer_health['health_status'] == 'Critical'
].nlargest(10, 'ltv')[['customer_id', 'ltv', 'health_score', 'health_status', 
                        'days_since_last_activity', 'risk_reason']]

print("🆘 Top 10 Critical Customers (by LTV):")
print(critical_customers.to_string(index=False))
print(f"\nTotal LTV at risk: ${critical_customers['ltv'].sum():,.2f}")

In [ ]:
# Plot Card 4.2: Top 10 Critical Customers (Bar Chart)
fig, ax = plt.subplots(figsize=(12, 6))

top_10 = critical_customers.sort_values('ltv', ascending=True).tail(10)
bars = ax.barh(range(len(top_10)), top_10['ltv'], color='#e74c3c', alpha=0.8)

# Add customer IDs as labels
ax.set_yticks(range(len(top_10)))
ax.set_yticklabels(top_10['customer_id'])

# Add LTV values on bars
for i, (idx, row) in enumerate(top_10.iterrows()):
    ax.text(row['ltv'] + 50, i, f"${row['ltv']:,.0f}", va='center', fontsize=9)

ax.set_title('Card 4.2: Top 10 Critical Customers (by LTV at Risk)', fontsize=14, fontweight='bold')
ax.set_xlabel('Lifetime Value ($)', fontsize=12)
ax.set_ylabel('Customer ID', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n💡 ACTION REQUIRED:")
print("   - Contact these customers TODAY to prevent churn")
print("   - Total potential revenue loss: ${:,.2f}".format(critical_customers['ltv'].sum()))

---
## Summary: All Numbers for BigQuery Verification

Compare these numbers with your BigQuery queries to ensure data accuracy.

In [ ]:
print("=" * 70)
print("DASHBOARD NUMBERS SUMMARY - Compare with BigQuery")
print("=" * 70)

print("\n📊 SECTION 1: Business Health Baseline")
print(f"  1. Current MRR: ${current_mrr:,.2f}")
print(f"  2. Churn Rate: {churn_rate:.2f}%")
print(f"  3. Active Customers: {active_customers:,}")
print(f"  4. Healthy Customers: {healthy_pct:.1f}%")

print("\n📈 SECTION 2: Growth Trajectory")
print(f"  - Latest MRR Growth: ${df_mrr_summary['total_mrr_growth'].iloc[-1]:,.2f}")
print(f"  - Latest Growth Rate: {df_mrr_summary['growth_rate_pct'].iloc[-1]:.2f}%")
print(f"  - Total data points: {len(df_mrr_summary)} months")

print("\n🎯 SECTION 3: Cohort Retention")
print(f"  - Total cohorts tracked: {len(cohorts)}")
print(f"  - Retention data points: {len(df_retention_12m)} (12 months × {len(cohorts)} cohorts)")
print(f"  - Churn heatmap cells: {len(df_churn_cohort)}")

print("\n🚨 SECTION 4: Customer Health")
print(f"  - Healthy: {health_distribution.get('Healthy', 0)} customers")
print(f"  - At Risk: {health_distribution.get('At Risk', 0)} customers")
print(f"  - Critical: {health_distribution.get('Critical', 0)} customers")
print(f"  - Total LTV at risk (top 10): ${critical_customers['ltv'].sum():,.2f}")

print("\n" + "=" * 70)
print("✅ If these numbers match your BigQuery queries, you're ready for Metabase!")
print("=" * 70)

---
## Next Steps

1. ✅ **Verify numbers** - Compare output above with BigQuery queries
2. ✅ **Connect BigQuery to Metabase** - Add database connection
3. ✅ **Run automation script** - `python3 day16_METABASE_auto_setup.py`
4. ✅ **Adjust visual styling** - Colors, fonts, layout in Metabase UI
5. ✅ **Take screenshots** - For documentation
6. ✅ **Export dashboard JSON** - Save final dashboard configuration